In [1]:
# Import libraries

import numpy as np
import pandas as pd
from pandasql import sqldf

In [2]:
psql = lambda q: sqldf(q, globals())

In [3]:
# Create employee data

employees = pd.DataFrame({'employee_id':[1,2,3,4,5],
                          'department_id':[101,101,101,102,103],
                          'boss_id':[3,3,0,5,3],
                          'name':['rob','bob','J','M','Mo'],
                          'salary':[30000,45000,65000,25000,67000]})
employees

,employee_id,department_id,boss_id,name,salary
0,1,101,3,rob,30000
1,2,101,3,bob,45000
2,3,101,0,J,65000
3,4,102,5,M,25000
4,5,103,3,Mo,67000


In [4]:
# Create department data

departments = pd.DataFrame({'department_id':[101,103,201,30,102],
                           'name':['IT','Docs','BI','DA','PM']})
departments

,department_id,name
0,101,IT
1,103,Docs
2,201,BI
3,30,DA
4,102,PM


In [5]:
# List employees (names) who have a bigger salary than their boss

q1 = (''' 
          SELECT e.name
          FROM employees AS e
          JOIN employees AS b
          ON e.boss_id = b.employee_id
          WHERE b.salary < e.salary

        ''')

psql(q1)

,name
0,Mo


In [6]:
# List employees who have the biggest salary IN their departments

q2 = ( 
       '''
           SELECT e.name,MAX(e.salary)
           FROM employees AS e
           JOIN departments AS d
           ON e.department_id = d.department_id
           GROUP BY e.department_id
           
       '''
     )

psql(q2)

,name,MAX(e.salary)
0,J,65000
1,M,25000
2,Mo,67000


In [7]:
# List departments that have less than 3 people in it

q3 = (
          '''
          SELECT d.name
          FROM employees AS e
          JOIN departments AS d
          ON e.department_id = d.department_id
          GROUP BY d.department_id
          HAVING COUNT(e.employee_id)<3
          '''
         )
psql(q3) 


,name
0,PM
1,Docs


In [8]:
# List all departments along with the number of people there 

q4 = (
    '''
    SELECT d.name, 
           COUNT(e.employee_id)  
    FROM employees AS e
    LEFT JOIN departments AS d
    ON e.department_id = d.department_id
    GROUP BY 1
    ORDER BY 1
    
    '''
     )

psql(q4)

,name,COUNT(e.employee_id)
0,Docs,1
1,IT,3
2,PM,1


In [9]:
# List employees that don't have a boss in the same department

q5 = (
    '''
    SELECT e1.name
    FROM employees AS e1
    JOIN employees AS e2
    ON e1.employee_id = e2.boss_id 
    WHERE e1.department_id <> e2.department_id
    '''
     )

psql(q5)

,name
0,J
1,Mo


In [10]:
# List all departments along with the total salary there

q6 = (
    '''
    SELECT e.department_id, 
    MAX(e.Salary)
    FROM employees AS e
    LEFT JOIN departments AS d
    ON e.department_id = d.department_id
    GROUP BY 1
    ORDER BY 1
    '''
     )

psql(q6)

,department_id,MAX(e.Salary)
0,101,65000
1,102,25000
2,103,67000
